In [73]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go

import pandas as pd

In [74]:
# importing weather data
# sam http://www.nerc-bas.ac.uk/icd/gjma/sam.html
# enso noaa
# iod https://psl.noaa.gov/gcos_wgsp/Timeseries/Data/dmi.had.long.data
enso = pd.read_csv("../weather_files/enso.csv")
enso.set_index('Year', inplace=True)

sam = pd.read_csv('../weather_files/sam.txt', sep='\s+')

iod = pd.read_csv('../weather_files/dmi.txt', sep='\s+')

In [75]:
enso_stack = enso.stack().reset_index()
enso_stack.columns = ['Year', 'Period', 'ENSO']

sam_stack = sam.stack().reset_index()
sam_stack.columns = ['Year', 'Period', 'SAM']

iod_stack = iod.stack().reset_index()
iod_stack.columns = ['Year', 'Period', 'IOD']

In [76]:
def smooth_data(df):
    # Create a datetime index
    df['Date'] = pd.to_datetime(df['Year'].astype(str) + df['Period'], format='%Y%b')
    df = df.set_index('Date')
    # Sort the index
    df = df.sort_index()
    
    # Calculate the 3-month rolling average
    smoothed = df.iloc[:, -1].rolling(window=3, center=True).mean()
    
    # Create a new dataframe with the smoothed data
    smoothed_df = pd.DataFrame({
        'Year': smoothed.index.year,
        'Period': smoothed.index.strftime('%b'),
        f'{df.columns[-1]}': smoothed.values
    })
    
    # Shift the Period names forward by one month
    month_shift = {
        'Jan': 'DJF', 'Feb': 'JFM', 'Mar': 'FMA', 'Apr': 'MAM', 'May': 'AMJ', 'Jun': 'MJJ',
        'Jul': 'JJA', 'Aug': 'JAS', 'Sep': 'ASO', 'Oct': 'SON', 'Nov': 'OND', 'Dec': 'NDJ'
    }
    smoothed_df['Period'] = smoothed_df['Period'].map(month_shift)
    
    return smoothed_df.dropna()

sam_smoothed = smooth_data(sam_stack)
iod_smoothed = smooth_data(iod_stack)

In [77]:
period_mapping = {
    'DJF': 'JAN',
    'JFM': 'FEB',
    'FMA': 'MAR',
    'MAM': 'APR',
    'AMJ': 'MAY',
    'MJJ': 'JUN',
    'JJA': 'JUL',
    'JAS': 'AUG',
    'ASO': 'SEP',
    'SON': 'OCT',
    'OND': 'NOV',
    'NDJ': 'DEC'
}

# Rename the periods in the merged_oni DataFrame
enso_stack['Period'] = enso_stack['Period'].replace(period_mapping)
iod_smoothed['Period'] = iod_smoothed['Period'].replace(period_mapping)
sam_smoothed['Period'] = sam_smoothed['Period'].replace(period_mapping)

In [78]:
merged_df = pd.merge(enso_stack, sam_smoothed, on=['Year', 'Period'], how='outer')
merged_df = pd.merge(merged_df, iod_smoothed, on=['Year', 'Period'], how='outer')

# Create a datetime column
merged_df['Date'] = pd.to_datetime(merged_df['Year'].astype(str) + '-' + merged_df['Period'], format='%Y-%b')

# Sort the DataFrame by date
merged_df = merged_df.sort_values('Date')

# Display the first few rows of the merged DataFrame
# print(merged_df.head())

merged_df.to_csv('climate_influencers.csv', index=False)

In [79]:
# Combine all datasets 
combined_df = pd.concat([
    enso_stack.assign(Metric='ENSO'),
    sam_smoothed.assign(Metric='SAM'),
    iod_smoothed.assign(Metric='IOD'),
])

# Create a datetime column
combined_df['Date'] = pd.to_datetime(combined_df['Year'].astype(str) + '-' + combined_df['Period'], format='%Y-%b')

# Create the Dash app
app = dash.Dash(__name__)

# Define custom CSS for the dropdown
custom_style = {
    'dropdown': {
        'font-family': 'Arial, sans-serif',
        'width': '200px'
    }
}

# Define color map with updated names
color_map = {
    'ENSO': '#00208F',
    'SAM': '#A90018',
    'IOD': '#1EBA9F',
    'Points': '#FA2449'
}

app.layout = html.Div([
    dcc.Graph(id='time-series-chart', style={'height': '80vh'}, config={'scrollZoom': False} ),
    html.Div([
        dcc.Dropdown(
            id='metric-dropdown',
            options=[{'label': i, 'value': i} for i in ['ENSO', 'SAM', 'IOD']],
            value='ENSO',
            style=custom_style['dropdown']
        )
    ], style={'position': 'absolute', 'top': '60px', 'right': '60px', 'zIndex': 1000})
])

@app.callback(
    Output('time-series-chart', 'figure'),
    Input('metric-dropdown', 'value')
)
def update_graph(selected_metric):
    filtered_df = combined_df[combined_df['Metric'] == selected_metric]
    
    fig = px.line(filtered_df, x='Date', y=selected_metric, title=f'{selected_metric} Over Time')
    
    # Update line color
    fig.update_traces(line=dict(color=color_map[selected_metric]))
    
    # Update x-axis to show only years
    fig.update_xaxes(
        dtick="M12",
        tickformat="%Y",
        ticklabelmode="period"
    )
    
    # Customize hover template
    fig.update_traces(
        hovertemplate="<b>Date</b>: %{x|%B %Y}<br><b>" + selected_metric + "</b>: %{y:.2f}<extra></extra>"
    )

    # Update layout for white background and other improvements
    fig.update_layout(
        plot_bgcolor='white',
        paper_bgcolor='white',
        title={
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(family="Arial, sans-serif")
        },
        font=dict(family="Arial, sans-serif"),
        xaxis=dict(
            showline=True,
            showgrid=False,
            showticklabels=True,
            linecolor='rgb(204, 204, 204)',
            linewidth=2,
            ticks='outside',
            tickfont=dict(
                family='Arial, sans-serif',
                size=12,
                color='rgb(82, 82, 82)',
            ),
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor='rgb(235, 235, 235)',
            showline=True,
            showticklabels=True,
            linecolor='rgb(204, 204, 204)',
            linewidth=2,
            ticks='outside',
            tickfont=dict(
                family='Arial, sans-serif',
                size=12,
                color='rgb(82, 82, 82)',
            ),
        ),
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=8052)

In [80]:
# combined_df